In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
# Set the path to your dataset directory
train_data_dir_cancer = 'Skin_Data/Cancer/Training'
test_data_dir_cancer = 'Skin_Data/Cancer/Testing'

train_data_dir_non_cancer = 'Skin_Data/Non_Cancer/Training'
test_data_dir_non_cancer = 'Skin_Data/Non_Cancer/Testing'

# Define image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32
 
# Data augmentation for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Rescale the testing dataset
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Generate training data for Cancer
train_generator_cancer = train_datagen.flow_from_directory(
    train_data_dir_cancer,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# Generate testing data for Cancer
test_generator_cancer = test_datagen.flow_from_directory(
    test_data_dir_cancer,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# Generate training data for Non_Cancer
train_generator_non_cancer = train_datagen.flow_from_directory(
    train_data_dir_non_cancer,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# Generate testing data for Non_Cancer
test_generator_non_cancer = test_datagen.flow_from_directory(
    test_data_dir_non_cancer,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# Combine generators for training data
train_generator = tf.keras.preprocessing.image.Iterator(
    train_generator_cancer, train_generator_non_cancer)

# Combine generators for testing data
test_generator = tf.keras.preprocessing.image.Iterator(
    test_generator_cancer, test_generator_non_cancer)

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=(len(train_generator_cancer) + len(train_generator_non_cancer)) // batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=(len(test_generator_cancer) + len(test_generator_non_cancer)) // batch_size)

# Evaluate the model
score = model.evaluate(test_generator)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: ('Invalid color mode:', <keras.src.preprocessing.image.DirectoryIterator object at 0x0000017FB43377D0>, '; expected "rgb", "rgba", or "grayscale".')